# Changement timestamp des sous-titres

Ce note book contient les fonctions pour modifier les timestamps des sous-titres
1. Ajouter du temps aux sous-titres (par ex. 3 secondes)
2. Ajouter du bruit dans des sous-titre, en suivant une Gaussienne

**Attention : utiliser des fichiers .vtt et changer le nom des dossiers dans la fonction `lister_fichiers` au besoin**

In [2]:
import re
from datetime import datetime, timedelta
import os 
import argparse
from collections import defaultdict,Counter
import json 
import csv
from tqdm import tqdm
from typing import List
import module as m
import numpy as np
from scipy.stats import norm

## Ajouter du temps aux sous-titres

In [2]:
# Matignon LSF - sous-titres d'origine
list_files = m.lister_fichiers("../data/cr_audio_aligned/") 

In [3]:
# Matignon LSF - sous titres segmentés en phrase
list_files = m.lister_fichiers("../data/cr_audio_aligned_sent_seg/")

In [4]:
seconde_to_add = 3.5

for files in list_files:
    # enregistrer les fichiers dans le dossier plus_one_sec/
    new_file = "cr_plus_3-5_sec/" + str(os.path.splitext(os.path.basename(files))[0]) +".vtt"
    with open(files,"r") as file:
        with open(new_file,"w") as output:
            content = ""
            line = file.readline()
            while line:
                # on prend la ligne qui correspond au timestamps
                if line.startswith("01:") or line.startswith("00:") or line.startswith("02:"):
                    timing_line = line.strip().split(' --> ')
                    start_time, end_time = timing_line
                    # et on modifie le timestamps à chaque fois
                    start_time = m.change_timecode(start_time,seconde_to_add)
                    end_time = m.change_timecode(end_time,seconde_to_add)
                    output.write(f"{start_time} --> {end_time}\n")
                else:
                    output.write(line)
                line = file.readline()

## Ajouter du bruit dans les timestamps des sous-titres

Code utilisé pour bruiter les sous-titres de Médiapi (en suivant plus ou moins les informations du corpus BOBSL sur le non alignement des sous-titres avec le signe), afin de pouvoir utiliser un transformer d'alignement des sous-titres (finetuning)

In [22]:
# Mediapi - sous titres d'origine
files_list = m.lister_fichiers("../data/aligned_mediapi/")


# Mediapi - sous titres segmentés en phrase
files_list = m.lister_fichiers("../data/mediapi_aligned_sent_seg/")

In [20]:
# Bruiter les sous-titres.
# Définir les paramètres de la distribution normale
moyenne = 0
ecart_type = 1
borne_inf = -5 # information BOBSL
borne_sup = 13 # information BOBSL

# Parcourt la liste des fichiers dans files_list
for file in files_list:
    # Construit le chemin pour le nouveau fichier VTT
    new_file = "non_aligned_mediapi_sent_seg/" + str(os.path.splitext(os.path.basename(file))[0]) + ".vtt"
    
    # Ouvre le fichier original en mode lecture
    with open(file, "r") as file:
        # Ouvre le nouveau fichier VTT en mode écriture
        with open(new_file, "w") as output:
            # Initialise une chaîne vide pour stocker le contenu modifié
            content = ""
            
            # Lit la première ligne du fichier original
            line = file.readline()
            last_end_time = None

            # Boucle pour parcourir chaque ligne du fichier original
            while line:
                try:
                    # Vérifie si la ligne commence par "00:", "01:", ou "02:"
                    if line.startswith("00:") or line.startswith("01:") or line.startswith("02:"):
                        # Générer une valeur aléatoire à partir de la distribution normale
                        valeur_normale = np.random.normal(loc=moyenne, scale=ecart_type)

                        # Ajuster la valeur pour respecter les bornes inférieure et supérieure
                        decalage_temporel = np.clip(valeur_normale, borne_inf, borne_sup)

                        # Divise la ligne de timing en deux parties : start_time et end_time
                        timing_line = line.strip().split(' --> ')
                        start_time, end_time = timing_line

                        # Convertit les chaînes de temps en objets de temps pour la vérification en dessous
                        start_time = m.convertir_chaine_en_temps(start_time)
                        end_time = m.convertir_chaine_en_temps(end_time)

                        # Vérifie si start_time est inférieur à last_end_time
                        if last_end_time is not None and start_time < last_end_time:
                            # Si c'est le cas, ajuste start_time pour qu'il soit égal à last_end_time
                            start_time = last_end_time

                        # Applique le décalage temporel aux temps de début et de fin
                        start_time = m.change_timecode(start_time, decalage_temporel)
                        end_time = m.change_timecode(end_time, decalage_temporel)

                        # Écrit les nouvelles lignes de timing dans le fichier de sortie
                        output.write(f"{start_time} --> {end_time}\n")

                        # Met à jour last_end_time avec la nouvelle valeur de end_time
                        last_end_time = end_time
                    else:
                        # Si la ligne ne commence pas par "00:", "01:", ou "02:", la copie telle quelle
                        output.write(line)
                except (OverflowError, ValueError) as e:
                    # Gère les erreurs OverflowError et ValueError, affiche un message et copie la ligne non modifiée
                    print(f"Erreur : {e}. Ligne non modifiée : {line} du fichier {file}")
                    output.write(line)
                    pass

                # Lit la ligne suivante du fichier original
                line = file.readline()



data/new_mediapi/67f46497f9.vtt non_aligned_mediapi_sent_seg/67f46497f9.vtt
data/new_mediapi/b41d730e81.vtt non_aligned_mediapi_sent_seg/b41d730e81.vtt
data/new_mediapi/0481a7f0af.vtt non_aligned_mediapi_sent_seg/0481a7f0af.vtt
data/new_mediapi/90d094256e.vtt non_aligned_mediapi_sent_seg/90d094256e.vtt
data/new_mediapi/b4823671dc.vtt non_aligned_mediapi_sent_seg/b4823671dc.vtt
data/new_mediapi/947de0ab18.vtt non_aligned_mediapi_sent_seg/947de0ab18.vtt
data/new_mediapi/7e92fc992c.vtt non_aligned_mediapi_sent_seg/7e92fc992c.vtt
data/new_mediapi/0ad3102bff.vtt non_aligned_mediapi_sent_seg/0ad3102bff.vtt
data/new_mediapi/60ab9aaef7.vtt non_aligned_mediapi_sent_seg/60ab9aaef7.vtt
data/new_mediapi/863dc1167b.vtt non_aligned_mediapi_sent_seg/863dc1167b.vtt
data/new_mediapi/a7756039fe.vtt non_aligned_mediapi_sent_seg/a7756039fe.vtt
data/new_mediapi/708db8d36a.vtt non_aligned_mediapi_sent_seg/708db8d36a.vtt
data/new_mediapi/152c562843.vtt non_aligned_mediapi_sent_seg/152c562843.vtt


OverflowError: date value out of range